In [27]:
import pandas as pd

df = pd.read_csv("all_datasets_aligned.csv")
print(df.head())          # Ver as primeiras linhas
print(df['flag'].value_counts())  # Quantos exemplos tem de cada classe


C:\Users\thelo\AppData\Local\Temp\ipykernel_1136\568968466.py:3: DtypeWarning: Columns (3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_datasets_aligned.csv")


    id  val1  val2 val3  val4   val5   val6   val7   val8    flag
0  790     5    33  104   9.0   33.0   33.0    0.0  111.0  BENIGN
1  399   254    91    0   0.0    0.0   60.0    0.0    0.0  BENIGN
2  608    25    33   34  48.0    8.0  142.0  109.0   58.0  BENIGN
3  672   100     0  154  29.0  151.0    2.0  189.0    0.0  BENIGN
4  809    64   187  127  20.0   17.0   32.0    0.0   20.0  BENIGN
flag
BENIGN    16189165
DoS         662184
RPM         654897
gear        597252
Fuzzy       491846
Name: count, dtype: int64


In [28]:
import pandas as pd

# Defina o número de amostras desejado para "BENIGN"
n_samples_benign = 1000_000  # ou 1_000_000, como preferir

# Separe a classe majoritária
benign_df = df[df['flag'] == 'BENIGN']

# Separe as demais classes (minoritárias)
minor_classes_df = df[df['flag'] != 'BENIGN']

# Amostragem aleatória da classe BENIGN
benign_sampled = benign_df.sample(n=n_samples_benign, random_state=42)

# Combine novamente o DataFrame balanceado
df_balanced = pd.concat([benign_sampled, minor_classes_df], axis=0)

# Embaralhar o DataFrame final
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Verificar a nova distribuição
print(df_balanced['flag'].value_counts())

df = df_balanced
#df.to_csv("all_datasets_aligned_balanced.csv", index=False)
del df_balanced, benign_df, minor_classes_df, benign_sampled

flag
BENIGN    1000000
DoS        662184
RPM        654897
gear       597252
Fuzzy      491846
Name: count, dtype: int64


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
df = df.dropna()

# Remove a coluna 'ID'
df = df.drop(columns=['id'])




In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Codificar rótulos em números
le = LabelEncoder()
df['flag'] = le.fit_transform(df['flag'])

X = df.drop(columns=['flag'])  # Features
y = df['flag']                 # Rótulos

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

del df  # Liberar memória

In [ ]:

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
# 1. Converter colunas para o tipo 'category'
for col in ['val3', 'val4', 'val6', 'val7']:
    X[col] = X[col].astype('category')

xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss',enable_categorical=True)

# -------------------------------------------------------------------------
# MUDANÇA: Grade de hiperparâmetros específica para o XGBoost
# -------------------------------------------------------------------------
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Validação cruzada estratificada (sem alterações aqui)
# OBS: n_splits=2 é baixo, considere usar 5 ou 10 para uma validação mais robusta.
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# -------------------------------------------------------------------------
# MUDANÇA: Passar o estimador 'xgb' para o GridSearchCV
# -------------------------------------------------------------------------
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, 
                           scoring='accuracy', cv=cv, n_jobs=-1, verbose=3) # n_jobs=-1 usa todos os cores

# Executa a busca
grid_search.fit(X, y)

# Resultados
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)

print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

# Para ver o desempenho do melhor modelo no dataset completo (ou em um set de teste)
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)
# print(classification_report(y_test, y_pred))




Fitting 2 folds for each of 48 candidates, totalling 96 fits
